<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/7_15_Multi_Armed_Bandit_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Armed Bandit

Exploitation(착취)와 Exploration(탐험)을 활용한 강화학습의 원리를 이용한 추천입니다.

실제 비즈니스에서 매우 많이 활용하는데, 카카오와 네이버에서도 이 방식을 많이 활용했다고 합니다.




## MAB 개요

여러개의 선택지 중 어떤 것을 선택할지 그 방식을 고르는 추천 알고리즘.  

카지노의 K개 슬롯머신을 N번 플레이 할 수 있는 상황. 각각의 슬롯머신은 당첨(1 or 0) 확률이 다를 때 수익을 최대화 하기 위한 선택의 순서를 찾는 것.



#### Exploration(탐험): 정보를 더 얻기 위해 계속해서 새로운 선택지를 선택하는 것
* 이것이 너무 적으면 더 잘 당첨될 수 있는 선택지를 포기하게 될 수 있습니다.
* 너무 많을 경우, reward에 비해 비용이 많이 나가게 됩니다.
#### Exploitation(착취): 현재까지의 기록으로 보아 가장 많이 나온 것을 선택하는 것
* 탐험과의 trade off가 있고, 이것을 최소화 하는 것이 MAB의 포인트입니다.







## MAB 문제

### MAB의 방식
* $q*(a) \dot= \mathbb{E}[R_t|A_t = a]$
  * $t$: 타임스텝 또는 횟수
  * $A_t$: 타임스텝 t에 진행한 행동 number (슬롯머신의 번호)
  * $R_t$: 타임스텝 t에 진행한 행동 $A_t$의 액션 $a$의 리워드(0, 1)
  * $q*(a)$: 이러한 액션 $a$의 reward의 실제 기대값

* 여기서 $q*(a)$를 추정할 때, 추정 가치가 최대인 action($a$)를 선택해 추정값 $Q_t(a)$를 정밀하게 하는 것
  * Greedy Action: 전체 선택지에 대해 n번 시행 중 가장 많이 reward 나온 선택지를 선택하는 것
    * Exploitation(착취)






### MAB 종류
#### 1. Simple Average Method (greedy action)
단순 평균 값을 통해 기댓값 $q*(a)$를 아주 간단하게 추정하는 방식
* 평균값에 따라 가장 추정가치가 높은 action을 선택하는 greedy algorithm
  * $A_t = argmaxQ_t(a)$

* $Q_t(a) \dot= \frac {타임스텝 t 이전에a의 reward의합}{타임스텝t이전의 a까지의 시도 수} = \frac {\sum_{i=1}^{t-1}R_i\cdot\mathbb{1}_A=a}{}$

### Epsilon greedy - 일반적으로 MAB에서 가장 기본이 되는 BASELINE 모델. 이것보다 항상 더 성능이 좋아야 한다.
* exploration이 적은 상태에서의 greedy action 방식을 조정한 것. 
* 동전을 예로 들어 앞면이 되면 Greedy, 뒷면이 나오면 랜덤 선택을 하게 되는 방식으로 이 때의 epsilon은 0.5로 설정이 됨. 일반적으로는 epsilon을 0.1~0.2로 설정해서 진행한다. 


### UCB(Upper Confidence Bound) - 특정 상태에서 선택이 적은 것을 찾아 선택 가능성을 높여줌
* $A_t \dot=argmax[Q_t(a) + c\sqrt {\frac {\text{ln}t}{N_t(a)}}]$
* $t: 타임 스텝$
* $Q_t(a)$: 타임스텝 $t$에서의 액션 $a$에 대한 예측된 reward (simple average)
* $N_t(a)$: 액션 $a$를 선택한 횟수 
* $c$: exploration을 조정하는 하이퍼파라미터
* 기본 추정치에 새로운 텀을 더해주는데, $\text{ln}t$로 인해 타임스텝이 증가하면 값 자체가 커지며, $a$가 분모에 있기 때문에 선택한 횟수가 적으면 그만큼 또 값이 커져 해당 액션에 대한 스코어가 커지게 됨
* 확률의 개념이 활용되지 않음 -> 기터비니스틱 하다고 하며, 특정 타임스텝$t$와 액션 $a$가 정해져 있기 때문에, 어느 순간이든 아웃풋이 정해져있음
* 결국 뒤의 텀을 통해 argmax를 취하면서 액션을 선택하다보면 어느순간 뒤의 텀은 작아지면서 기본적인 평균 $Q_t(a)$에 수렴하게 된다.


### MAB를 활용한 추천
* MAB의 액션이 곧 추천 아이템이 됨.
  * ex:  추천 아이템 100개 중에 3번을 추천하는 것 -> 3번 액션을 선택
* Reward: 유저가 클릭을 하는지 안하는지 = CTR. 100번 노출해서 10번 클릭을 유도했다면, CTR은 10%가 됨

* MF는 사용자 또는 아이템에 대한 스코어를 구하는 것인데, MAB는 CTR을 maximising하는 것을 목표로 함

* 단순 MAB만 한다면 개인화 추천 자체는 힘들 수 있음. 사람에 따라 reward가 다를 것이기 때문에 클러스터링 등을 통해서 사용자를 그룹화 하는 방법도 있음

### 톰슨 샘플링(Thompson Sampling)

* epsilon greedy: 가장 좋은 것 혹은 epsilon 확률로 random sampling 
* UCB: 기본 액션 뒤에 불확실성 term을 두어서 별로 안 뽑힌 액션이 선택될 수 있도록 설정
* 톰슨: 위 두가지 방법과는 달리 주어진 action들에 해당하는 확률분포를 구해서 해결. 데이터가 많을 경우는 확률분포가 확실하기 때문에 톰슨이 좀 더 유리해질 수 있음.(베타분포 활용)

#### 베타분포

* 두 개의 양의 매개변수 $\alpha$와 $\beta$로 표현되며, 



#### 톰슨 샘플링 원리
k개의 선택지에 대해 각각 베타분포를 적용
  * 각 액션 별로 베타분포를 구해 


